In [1]:
!nvidia-smi

Mon Mar  4 17:52:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [3]:
info = []
for letter in range(ord('a'), ord('z') + 1):
    url = 'https://www.medicineindia.org/medicine-generics/'+chr(letter)
    info.append(url)

In [4]:
soup_list = []
for i in info:
    res = requests.get(i)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        soup_list.append(soup)
    else:
        print(f"Failed to fetch {url}. Status code: {res.status_code}")

In [5]:
table_list = []
for i in range (len(soup_list)):
    table = soup_list[i].find('table',class_='table table-striped')
    table_list.append(table)

In [6]:
gen_list = []
for i in range(len(table_list)):
    for row in table_list[i].find_all('tr', class_='generic-medicine-row'):
        gen_list.append(row.text)

In [7]:
import re

def split_string_at_capital(string):
    parts = re.split(r'(?<=[0-9a-zA-Z\)\]])[A-Z]', string)
    if len(parts) > 1:
        return parts[0]
    else:
        return string

In [8]:
column1 = []
for i in gen_list:
    res = split_string_at_capital(i)
    column1.append(res)

In [9]:
Brands = []
for i in range(len(table_list)):
    for row in table_list[i].find_all('a'):
        Brands.append(row.text)

In [10]:
column2 = Brands[::2]
column3 = Brands[1::2]

In [11]:
df=pd.DataFrame({
    'Name':column1,
    'Brand':column2,
    'Info':column3
    })

In [12]:
df.to_excel('GM Info.xlsx', index=False)

In [13]:
Brands_links = []
for i in range(len(table_list)):
    for row in table_list[i].find_all('a'):
        Brands_links.append(row.get('href'))

In [14]:
links  = Brands_links[::2]

In [15]:
brand_link = ["https:" + url for url in links]

In [16]:
header_data = []
new = []
count = 0
for i,url in enumerate (brand_link,start=0):
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'lxml')
    target_table = soup.find('table', class_='table table-striped table-bordered')

    if target_table is not None:
        for row in target_table.find_all('td'):
            header_data.append(row.text)
            new.append(column1[i])
    else:
      count = count+1
      print(f"No table found on the webpage: {url}")

No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2856/acetic-acid
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2722/adalimumab
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2896/adenosylcobalamin
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/1454/agar
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/1087/albuterol
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2001/alclometasone-dipropionate
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2989/alfacalcidol-beta-carotene-magnesium-oxide
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/1427/alginic-acid
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/2697/aliskiren
No table found on the webpage: https://www.medicineindia.org/brands-for-generic/3378/a

In [21]:
Brand = header_data[::5]
Company = header_data[1::5]
Package = header_data[2::5]
Strength = header_data[3::5]
Price = header_data[4::5]

Names = new[::5]

In [31]:
df2=pd.DataFrame({
    'Name': Names,
    'Brand': Brand,
    'Company': Company,
    'Package' : Package,
    'Strength' : Strength,
    'Price' : Price
    })

In [31]:
df2

In [31]:
df2.to_excel('Generic Medicine.xlsx', index=False)